<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-01 09:47:15
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  1.57 L
Current:  1.39 C
-------------------
Today PnL: 68.99 K (0.5%)
Current PnL: -22.13 L (-14.69%)
CY Booked + Current PnL: -10.69 L (-7.09%)
-------------------
Total profit:  1.16 L
Total loss:  -23.28 L
-------------------
Total Booked + Current PnL: 16.29 L (13.17%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.2%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.28 C
Est FTT PnL: 88.50 L (63.46%)
Deployed:  1.24 C
Current:  1.39 C
CAGR/XIRR %: 7.33%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.52,113192.0,12688.0,16424.0,-0.14,12.62,14.51,28.96,79.0,0.77,0.82,50.88,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.81,37.0,M-SC,2.45,83467.0,-17310.0,17403.0,0.35,-17.18,20.85,0.09,245.0,-0.99,0.61,9.66,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,8.25,43.0,H-MC,3.38,186760.0,22090.0,21235.0,-1.48,13.41,11.37,26.31,89.0,1.04,1.35,22.47,X40N,NTT,AC
57,RELIANCE,1533.00,-14.41,45.0,H-LC,3.71,214531.0,4165.0,24607.0,0.82,1.98,11.47,13.68,37.0,0.17,1.56,18.83,XY25,NTT,REFINERIES
42,ITC,452.00,-39.19,41.0,H-LC,2.08,196187.0,-3951.0,24837.0,-0.09,-1.97,12.66,10.44,4.0,-0.16,1.42,3.48,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,71.05,78.0,H-SC,1.80,157029.0,-22042.0,87764.0,15.85,-12.31,55.89,36.71,135.0,-0.25,1.14,55.80,XR,NTT,BANKS
64,SHALBY,327.00,1238.12,69.0,M-SC,14.25,181947.0,804.0,44668.0,6.24,0.44,24.55,25.10,235.0,0.02,1.32,45.81,XY24,NTT,HEALTHCARE
26,GREENPANEL,537.00,213.95,50.0,M-SC,1.18,140056.0,-43022.0,122003.0,3.99,-23.50,87.11,43.14,230.0,-0.35,1.02,31.74,XY24,NTT,MISC
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,40.37,79968.0,-33581.0,73635.0,3.02,-29.57,92.08,35.27,268.0,-0.46,0.58,95.72,XR,NTT,HOTELS
52,PGHH,17973.08,-27.64,63.0,H-MC,3.21,209625.0,8805.0,59974.0,2.40,4.38,28.61,34.25,80.0,0.15,1.52,9.81,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-47.86,23.0,M-SC,1.19,88072.0,-3426.0,82180.0,-4.73,-3.74,93.31,86.07,223.0,-0.04,0.64,27.90,XR,NTT,DURABLES
55,RAJOOENG,143.10,-45.85,20.0,H-SC,14.28,82110.0,-20390.0,60991.0,-3.99,-19.89,74.28,39.61,114.0,-0.33,0.60,0.00,AR,ATH,MISC
81,VAIBHAVGBL,521.00,48.21,44.0,H-SC,5.80,128349.0,-54426.0,165493.0,-2.11,-29.78,128.94,60.77,125.0,-0.33,0.93,14.96,XR,NTT,JEWELLERY
3,ALKYLAMINE,4546.37,-10.73,31.0,H-SC,10.71,83029.0,-17934.0,112463.0,-1.99,-17.76,135.45,93.63,148.0,-0.16,0.60,23.67,SR,ATH,CHEMICALS
38,INDIGOPNTS,1408.00,102.57,30.0,M-SC,1.51,132680.0,-41879.0,41914.0,-1.89,-23.99,31.59,0.02,221.0,-1.00,0.96,15.41,OX40N,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1238.12,69.0,M-SC,14.25,181947.0,804.0,44668.0,6.24,0.44,24.55,25.10,235.0,0.02,1.32,45.81,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-185.23,79.0,M-SC,8.19,147780.0,-1440.0,145622.0,2.08,-0.97,98.54,96.62,208.0,-0.01,1.07,59.93,XY25,NTT,FINANCE
37,INDIAMART,4850.92,-55.76,23.0,H-SC,9.96,123320.0,-16.0,133778.0,-1.69,-0.01,108.48,108.45,119.0,-0.00,0.89,22.34,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,99.53,58.0,H-SC,11.27,125685.0,-16794.0,36310.0,0.43,-11.79,28.89,13.70,163.0,-0.46,0.91,48.34,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.81,37.0,M-SC,2.45,83467.0,-17310.0,17403.0,0.35,-17.18,20.85,0.09,245.0,-0.99,0.61,9.66,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,51.0,H-SC,1.54,221310.0,-48357.0,84695.0,-0.28,-17.93,38.27,13.47,138.0,-0.57,1.60,11.94,XY24,NTT,PAINTS
18,CERA,9475.0,-22.91,35.0,H-SC,1.61,140656.0,-35247.0,77262.0,-0.70,-20.04,54.93,23.89,149.0,-0.46,1.02,21.48,OX40N,NTT,CERAMICS
66,SIS,528.0,2099.48,44.0,H-SC,4.04,88753.0,-22279.0,45885.0,0.83,-20.07,51.70,21.26,156.0,-0.49,0.64,19.91,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1238.12,69.0,M-SC,14.25,181947.0,804.0,44668.0,6.24,0.44,24.55,25.10,235.0,0.02,1.32,45.81,XY24,NTT,HEALTHCARE
60,SAIL,228.00,48.36,61.0,M-MC,11.55,234020.0,9058.0,158595.0,1.06,4.03,67.77,74.53,192.0,0.06,1.70,37.58,XY24,BTT,STEEL
32,HINDZINC,730.22,29.16,66.0,M-LC,3.90,210823.0,5747.0,106824.0,0.46,2.80,50.67,54.89,52.0,0.05,1.53,27.57,X5K,ATH,METALS
14,BLUESTARCO,2080.00,8.25,43.0,H-MC,3.38,186760.0,22090.0,21235.0,-1.48,13.41,11.37,26.31,89.0,1.04,1.35,22.47,X40N,NTT,AC
84,VOLTAS,1530.00,-2.62,45.0,H-MC,2.86,204240.0,12498.0,25264.0,1.60,6.52,12.37,19.69,99.0,0.49,1.48,13.76,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.52,113192.0,12688.0,16424.0,-0.14,12.62,14.51,28.96,79.0,0.77,0.82,50.88,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,66.0,H-MC,9.40,190174.0,10366.0,102047.0,0.19,5.77,53.66,62.52,88.0,0.10,1.38,41.82,XR,NTT,BANKS
32,HINDZINC,730.22,29.16,66.0,M-LC,3.90,210823.0,5747.0,106824.0,0.46,2.80,50.67,54.89,52.0,0.05,1.53,27.57,X5K,ATH,METALS
37,INDIAMART,4850.92,-55.76,23.0,H-SC,9.96,123320.0,-16.0,133778.0,-1.69,-0.01,108.48,108.45,119.0,-0.00,0.89,22.34,AR,ATH,MISC
86,WIPRO,420.00,-15.61,38.0,M-LC,5.80,148403.0,-2542.0,112000.0,-0.00,-1.68,75.47,72.51,53.0,-0.02,1.08,4.24,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JIOFIN,387.00,-7.23,25.0,H-LC,13.01,252152.0,-15213.0,80663.0,-0.34,-5.69,31.99,24.48,48.0,-0.19,1.83,46.15,XY24,BTT,FINANCE
5,ASIANPAINT,3465.66,-18.55,28.0,H-LC,5.83,205489.0,-46279.0,99498.0,-0.63,-18.38,48.42,21.13,27.0,-0.47,1.49,10.53,X40,ATH,PAINTS
74,TCS,4476.75,-31.90,28.0,H-LC,12.42,271575.0,-74381.0,149230.0,0.02,-21.50,54.95,21.64,1.0,-0.50,1.97,0.02,X40,ATH,IT
30,HCLTECH,1943.91,-4.09,35.0,H-LC,10.23,218560.0,-23364.0,86637.0,0.51,-9.66,39.64,26.15,8.0,-0.27,1.59,3.28,X40,ATH,IT
83,VBL,671.64,-20.80,35.0,H-LC,9.23,285026.0,-30816.0,143482.0,0.69,-9.76,50.34,35.67,5.0,-0.21,2.07,2.75,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.51,42.0,H-LC,1.59,229320.0,-16831.0,96039.0,-1.20,-6.84,41.88,32.18,16.0,-0.18,1.66,26.74,X200,ATH,IT
65,SIEMENS,4671.5,-1.97,47.0,H-LC,1.96,157615.0,-28480.0,75955.0,0.74,-15.30,48.19,25.51,15.0,-0.37,1.14,16.15,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.19,41.0,H-LC,2.08,196187.0,-3951.0,24837.0,-0.09,-1.97,12.66,10.44,4.0,-0.16,1.42,3.48,X40,NTT,FMCG
57,RELIANCE,1533.0,-14.41,45.0,H-LC,3.71,214531.0,4165.0,24607.0,0.82,1.98,11.47,13.68,37.0,0.17,1.56,18.83,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-22.87,39.0,H-LC,3.99,205623.0,1123.0,40282.0,0.87,0.55,19.59,20.25,10.0,0.03,1.49,9.33,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.90,28.0,H-LC,12.42,271575.0,-74381.0,149230.0,0.02,-21.50,54.95,21.64,1.0,-0.50,1.97,0.02,X40,ATH,IT
83,VBL,671.64,-20.80,35.0,H-LC,9.23,285026.0,-30816.0,143482.0,0.69,-9.76,50.34,35.67,5.0,-0.21,2.07,2.75,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,35.0,H-LC,10.23,218560.0,-23364.0,86637.0,0.51,-9.66,39.64,26.15,8.0,-0.27,1.59,3.28,X40,ATH,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.08,196187.0,-3951.0,24837.0,-0.09,-1.97,12.66,10.44,4.0,-0.16,1.42,3.48,X40,NTT,FMCG
40,INFY,2275.00,-21.39,37.0,H-LC,8.20,305783.0,-7105.0,178791.0,-0.43,-2.27,58.47,54.87,3.0,-0.04,2.22,4.23,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-1.97,47.0,H-LC,1.96,157615.0,-28480.0,75955.0,0.74,-15.30,48.19,25.51,15.0,-0.37,1.14,16.15,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,41.0,H-LC,2.08,196187.0,-3951.0,24837.0,-0.09,-1.97,12.66,10.44,4.0,-0.16,1.42,3.48,X40,NTT,FMCG
19,CIPLA,1795.00,-22.87,39.0,H-LC,3.99,205623.0,1123.0,40282.0,0.87,0.55,19.59,20.25,10.0,0.03,1.49,9.33,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.55,28.0,H-LC,5.83,205489.0,-46279.0,99498.0,-0.63,-18.38,48.42,21.13,27.0,-0.47,1.49,10.53,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,45.0,H-LC,3.71,214531.0,4165.0,24607.0,0.82,1.98,11.47,13.68,37.0,0.17,1.56,18.83,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.90,28.0,H-LC,12.42,271575.0,-74381.0,149230.0,0.02,-21.50,54.95,21.64,1.0,-0.50,1.97,0.02,X40,ATH,IT
40,INFY,2275.00,-21.39,37.0,H-LC,8.20,305783.0,-7105.0,178791.0,-0.43,-2.27,58.47,54.87,3.0,-0.04,2.22,4.23,X40,BTT,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.08,196187.0,-3951.0,24837.0,-0.09,-1.97,12.66,10.44,4.0,-0.16,1.42,3.48,X40,NTT,FMCG
83,VBL,671.64,-20.80,35.0,H-LC,9.23,285026.0,-30816.0,143482.0,0.69,-9.76,50.34,35.67,5.0,-0.21,2.07,2.75,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,44.0,H-LC,7.70,248904.0,-12715.0,131919.0,0.23,-4.86,53.00,45.57,7.0,-0.10,1.81,5.91,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,40.37,79968.0,-33581.0,73635.0,3.02,-29.57,92.08,35.27,268.0,-0.46,0.58,95.72,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,41.0,H-SC,12.38,91500.0,-6480.0,28081.0,0.78,-6.61,30.69,22.05,152.0,-0.23,0.66,33.00,XR,ATH,FINANCE
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.52,113192.0,12688.0,16424.0,-0.14,12.62,14.51,28.96,79.0,0.77,0.82,50.88,MH,ATH,METALS
78,UJJIVANSFB,60.00,99.53,58.0,H-SC,11.27,125685.0,-16794.0,36310.0,0.43,-11.79,28.89,13.70,163.0,-0.46,0.91,48.34,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,79.0,M-SC,8.19,147780.0,-1440.0,145622.0,2.08,-0.97,98.54,96.62,208.0,-0.01,1.07,59.93,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1238.12,69.0,M-SC,14.25,181947.0,804.0,44668.0,6.24,0.44,24.55,25.10,235.0,0.02,1.32,45.81,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-185.23,79.0,M-SC,8.19,147780.0,-1440.0,145622.0,2.08,-0.97,98.54,96.62,208.0,-0.01,1.07,59.93,XY25,NTT,FINANCE
69,SURYODAY,240.00,71.05,78.0,H-SC,1.80,157029.0,-22042.0,87764.0,15.85,-12.31,55.89,36.71,135.0,-0.25,1.14,55.80,XR,NTT,BANKS
60,SAIL,228.00,48.36,61.0,M-MC,11.55,234020.0,9058.0,158595.0,1.06,4.03,67.77,74.53,192.0,0.06,1.70,37.58,XY24,BTT,STEEL
50,NATIONALUM,244.55,-43.66,62.0,H-MC,12.52,113192.0,12688.0,16424.0,-0.14,12.62,14.51,28.96,79.0,0.77,0.82,50.88,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.10
1,25,43.46
2,50,74.91


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.11
LC    32.34
MC    23.56
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.39
X40      14.60
X40N     11.33
XR       11.18
XY25     10.55
AR        8.14
OX40N     7.82
X200      1.66
MH        1.63
X5K       1.53
SR        1.18
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.15
H-LC    25.71
H-MC    20.58
M-SC    13.51
M-LC     5.58
M-MC     2.68
L-SC     1.45
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.93,-6.89,42.17
IT,12.55,-21.53,87.79
FINANCE,10.43,-17.20,70.43
MISC,6.96,-19.82,86.43
BANKS,6.47,-11.84,70.67
PAINTS,5.57,-20.14,38.05
ELECTRICAL,5.44,-9.69,48.17
INSURANCE,3.76,-6.49,46.30
AC,3.53,0.81,15.80


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3195213.0,22
XR,1367132.0,14
AR,1210766.0,9
X40,835465.0,10
X40N,640970.0,9
OX40N,567428.0,10
XY25,482252.0,7
SR,258425.0,2
X5K,106824.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3770289.0,29
M-SC,1484099.0,17
H-LC,1276535.0,15
H-MC,1267790.0,15
M-LC,381731.0,4
M-MC,309124.0,2
L-SC,260572.0,3
L-MC,60671.0,1
L-LC,39319.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1238811.0      6
           AR         832852.0      5
M-SC       XY24       819484.0      7
H-SC       XR         789706.0      7
H-MC       XY24       584224.0      4
H-LC       X40        538993.0      5
H-SC       X40N       316073.0      4
M-MC       XY24       309124.0      2
H-SC       OX40N      261230.0      4
           SR         258425.0      2
H-LC       X40N       235530.0      3
H-MC       X40        221113.0      4
H-LC       AR         207874.0      2
H-MC       XY25       179875.0      2
L-SC       XR         171870.0      2
M-SC       AR         170040.0      2
M-LC       XY24       162907.0      2
M-SC       XY25       145622.0      1
           OX40N      142756.0      4
           XR         130838.0      2
H-LC       XY25       117436.0      3
M-LC       XR         112000.0      1
           X5K        106824.0      1
H-MC       XR         102047.0      1
H-LC       X200        96039.0      1
H-MC       X40N        89367.0      2
L-SC       OX40N       88702.0      1
H-LC       XY24        80663.0      1
M-SC       X40         75359.0      1
H-MC       OX40N       74740.0      1
H-SC       MH          73192.0      1
L-MC       XR          60671.0      1
L-LC       XY25        39319.0      1
H-MC       MH          16424.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 51.0 seconds
